# Проект по SQL
# Анализ базы данных сервиса для чтения книг

**Цели исследования:**

Сформировать ценностное предложения для нового продукта, для этого необходимо:
* Посчитать количество книг, выпущенных в 21 веке, количество обзоров на них и среднюю оценку;
* Определить самое выпускающее издательство;
* Определить лучшего автора на основании средней оценки его книг;
* Посчитать сколько обзоров в среднем оставляют самые активные пользователи.


**Задачи:**
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
5. Посчитайть среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Описание данных:**

Таблица `books`. Содержит данные о книгах:
* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

Таблица `authors`. Содержит данные об авторах:
* `author_id` — идентификатор автора;
* `author` — имя автора.

Таблица `publishers`. Содержит данные об издательствах:
* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства;

Таблица `ratings`. Содержит данные о пользовательских оценках книг:
* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

Таблица `reviews`. Содержит данные о пользовательских обзорах:
* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя автора обзора;
* `text` — текст обзора.

## Загрузка модулей и данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': '***', # имя пользователя
'pwd': '***', # пароль
'host': '***',
'port': 6432, # порт подключения
'db': '***'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# Вывод первых пяти строк таблицы books
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# Вывод первых пяти строк таблицы reviews
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [5]:
# Вывод первых пяти строк таблицы ratings
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# Вывод первых пяти строк таблицы publishers
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
# Вывод первых пяти строк таблицы authors
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


## Выполнение задач
### Книги, выпущенные после 1 января 2020 года

In [8]:
# Количество книг, выпущенных после 1 января 2000 года
query = '''SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01';'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


В базе данных сервиса находится 819 книг, выпущенных после 1 января 2000 года.

### Исследование книг

In [9]:
# Количество обзоров и средняя оценка каждой книги
query = '''SELECT b.book_id,
b.title,
COUNT(DISTINCT review_id),
AVG(rating)
FROM books b
LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
GROUP BY b.book_id
ORDER BY COUNT(review_id) DESC;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count,avg
0,948,Twilight (Twilight #1),7,3.662500
1,750,The Hobbit or There and Back Again,6,4.125000
2,673,The Catcher in the Rye,6,3.825581
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
...,...,...,...,...
995,221,Essential Tales and Poems,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


В результате получилась таблица, в которой: `count` - количество обзоров книги, `avg` - средняя оценка книги.

Самой обозревамой книгой стала первая часть Сумерок (Twilight) - 7 обзоров со средней оценкой 3.66. На втором месте книга про Хоббитов (The Hobbit or There and Back Again) с 6 отзывами и оценкой 4.13. Почётное третье место заняла книга "Над пропастью во ржи" (The Catcher in the Rye), у которой тоже 6 отзывов со средней оценкой 3.83.

### Изучение издательств

In [10]:
# Определение самого выпускающего издательства
query = '''SELECT p.publisher,
COUNT(book_id)
FROM books b
LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages >= 50
GROUP BY p.publisher
ORDER BY COUNT(book_id) DESC
LIMIT 1;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42


Издательство `Penguin Books` выпустило наибольшее число книг (42 штуки), толще 50 см.

### Составление рейтинга авторов

In [11]:
# Определение автора с самой высокой оценкой
query = '''SELECT a.author,
AVG(rat.rating),
COUNT(rat.rating) AS count_rating
FROM authors AS a
INNER JOIN books AS b ON b.author_id = a.author_id
INNER JOIN ratings AS rat ON rat.book_id = b.book_id
GROUP BY a.author
HAVING COUNT(rat.rating) >= 50 
ORDER BY count_rating DESC
LIMIT 1;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg,count_rating
0,J.K. Rowling/Mary GrandPré,4.288462,312


Автором с самой высокой оценкой книги является дуэт `J.K. Rowling/Mary GrandPré`. Их книги оценили 312 пользователей со средней оценкой 4.29.

### Расчет среднего количества обзоров книг от пользователей

In [12]:
# Расчет среднего количества обзоров от пользователей
query = '''SELECT AVG(num_reviews) AS avg_reviews
FROM (SELECT COUNT(review_id) AS num_reviews
  FROM reviews
  WHERE username IN (SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 48)
  GROUP BY username) AS sub;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_reviews
0,24.0


Пользователи, которые поставили больше 48 оценок, создали в среднем по 24 обзора.

## Выводы

В ходе проекта была проанализирована база данных сервиса для чтения книг, которая состоит из 5 таблиц.

В результате было определено, что:
* С 1 января 2000 года было выпущено 819 книг;
* Самой обозревамой книгой стала первая часть Сумерок (Twilight) - 7 обзоров со средней оценкой 3.66;
* Самым выпускающим стало издательство `Penguin Books`, которое опубликовало 42 книги;
* Автор с самой высокой оценкой книги - `J.K. Rowling/Mary GrandPré`. Их книги оценили 312 пользователей со средней оценкой 4.29;
* Самые активные пользователи оставили в среднем по 24 обзора.